# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [26]:
#!cat publications.csv

## Import pandas

We are using the very handy pandas library for dataframes.

In [10]:
import pandas as pd
import os
import re

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [43]:
conv = {'publi_id': lambda x: str(x), 'page': lambda x: str(x), 'volume': lambda x: str(x), 'arxiv_id': lambda x: str(x)}
publications = pd.read_csv("publications.csv", sep=";", header=0, converters=conv)#, engine ='python')
publications

,pub_date,title,authors,venue,volume,page,excerpt,url_slug,paper_url,arxiv_id,doi,tags
0,2013-06-04,"Hysteresis, phase transitions, and dangerous t...","C. Duclut, S. Backhaus, and M. Chertkov",Phys. Rev. E,87,62802,The majority of dynamical studies in power sys...,duclut2013hysteresis,https://link.aps.org/doi/10.1103/PhysRevE.87.0...,1212.0252,10.1103/PhysRevE.87.062802,"smartGrid, phaseTransition"
1,2016-09-02,Langevin Equations for Reaction-Diffusion Proc...,"F. Benitez, C. Duclut, H. Chaté, B. Delamotte,...",Phys. Rev. Lett.,117,100601,For reaction-diffusion processes with at most ...,benitez2016langevin,https://link.aps.org/doi/10.1103/PhysRevLett.1...,1604.06740,10.1103/PhysRevLett.117.100601,statisticalPhysics
2,2017-01-05,Frequency regulators for the nonperturbative r...,"C. Duclut, and B. Delamotte",Phys. Rev. E,95,12107,We derive the necessary conditions for impleme...,duclut2017frequency,http://link.aps.org/doi/10.1103/PhysRevE.95.01...,1611.07301,10.1103/PhysRevE.95.012107,"statisticalPhysics, phaseTransition, renormali..."
3,2017-07-27,Nonuniversality in the erosion of tilted lands...,"C. Duclut, and B. Delamotte",Phys. Rev. E,96,12149,The anisotropic model for landscapes erosion p...,duclut2017nonuniversality,http://link.aps.org/doi/10.1103/PhysRevE.96.01...,1705.05294,10.1103/PhysRevE.96.012149,"statisticalPhysics, phaseTransition, renormali..."
4,2019-09-24,Fluid pumping and active flexoelectricity can ...,"C. Duclut, N. Sarkar, J. Prost, and F. Jülicher",Proc. Natl. Acad. Sci. U.S.A.,116,19264,We discuss the physical mechanisms that promot...,duclut2019fluid,http://www.pnas.org/lookup/doi/10.1073/pnas.19...,1905.08023,10.1073/pnas.1908481116,"tissues, electrohydraulics, hydraulics"
5,2021-01-29,Nonequilibrium polarity-induced chemotaxis: Em...,"S. Mahdisoltani, R. Ben Alì Zinati, C. Duclut,...",Phys. Rev. Research,3,13100,A generically observed mechanism that drives t...,mahdisoltani2021nonequilibrium,https://link.aps.org/doi/10.1103/PhysRevResear...,1911.08115,10.1103/PhysRevResearch.3.013100,"statisticalPhysics, phaseTransition, renormali..."
6,2021-05-11,Hydraulic and electric control of cell spheroids,"C. Duclut, J. Prost, and F. Jülicher",Proc. Natl. Acad. Sci. U.S.A.,118,e2021972118,We use a theoretical approach to examine the e...,duclut2021hydraulic,http://www.pnas.org/lookup/doi/10.1073/pnas.20...,2011.03735,10.1073/pnas.2021972118,"tissues, electrohydraulics, hydraulics"
7,2021-09-27,Nonlinear rheology of cellular networks,"C. Duclut, J. Paijmans, M. M. Inamdar, C. D. M...",Cells & Development,168,203746,Morphogenesis depends crucially on the complex...,duclut2021nonlinear,https://doi.org/10.1016/j.cdev.2021.203746,2103.16462,10.1016/j.cdev.2021.203746,"tissues, mechanics, rheology, vertexModel"
8,2022-03-11,Coarse-grained curvature tensor on polygonal s...,"C. Duclut, A. Amiri, J. Paijmans, and F. Jülicher",SciPost Phys. Core,5,11,"Using concepts from integral geometry, we prop...",duclut2022coarse-grained,https://scipost.org/10.21468/SciPostPhysCore.5...,2104.07988,10.21468/SciPostPhysCore.5.1.011,"integralGeometry, tissues"
9,2022-05-26,Passive odd viscoelasticity,"R. Lier, J. Armas, S. Bo, C. Duclut, F. Jülich...",Phys. Rev. E,105,54607,Active chiral viscoelastic materials exhibit e...,lier2022passive,https://link.aps.org/doi/10.1103/PhysRevE.105....,2109.06606,10.1103/PhysRevE.105.054607,"oddElasticity, oddViscosity, rheology, thermod..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [44]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

def identify_id(id_str):
    arxiv_regex = re.compile(r'^\d{4}\.\d{5}$') # Regex for arXiv ID (e.g., 2402.05078)
    biorxiv_regex = re.compile(r'^\d{4}\.\d{2}\.\d{2}\.\d{6}$') # Regex for bioRxiv ID (e.g., 2023.12.23.573034)
    if biorxiv_regex.match(id_str):
        return "biorxiv"
    elif arxiv_regex.match(id_str):
        return "arxiv"
    else:
        return "unknown"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [69]:
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
#     md_filename = item.url_slug + ".md"
#     html_filename = item.url_slug
    year = item.pub_date[:4]
    
    pub_number = str(row+1)

    preprint_type = identify_id(item.arxiv_id)
    
    if (item.venue=='preprint'):
        if preprint_type=="biorxiv":
            citation = '"' + item.title + '", ' + item.authors + \
            ', <i>bioRxiv:' + item.arxiv_id + '</i>' + \
            ' (' + str(year) + ').'
        if preprint_type=="arxiv":
            citation = '"' + item.title + '", ' + item.authors + \
            ', <i>arXiv:' + item.arxiv_id + '</i>' + \
            ' (' + str(year) + ').'
    else:
        citation = '"' + item.title + '", ' + item.authors + \
        ', <i>' + item.venue + '</i> <b>' + \
        str(item.volume) + '</b>, ' + str(item.page) \
        + ' (' + str(year) + ').'
        
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    md += "\nurlslug: '" + item.url_slug + "'"
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nauthors: '" + item.authors + "'"
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if preprint_type=="biorxiv":
        md += "\npreprintvenue: 'bioRxiv'" 
    elif preprint_type=="arxiv":
        md += "\npreprintvenue: 'arXiv'" 
    else:
        md += "\npreprintvenue: 'none'" 
    
    md += "\nvolume: '" + str(item.volume) + "'"
    
    md += "\npage: '" + str(item.page) + "'"
    
    if (item.paper_url=='/'):
        if preprint_type=="biorxiv":
            md += "\npaperurl: 'https://www.biorxiv.org/content/10.1101/" + item.arxiv_id + "'"
        elif preprint_type=="arxiv":
            md += "\npaperurl: 'https://arxiv.org/abs/" + item.arxiv_id + "'"
    else:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\narxivid: '" + item.arxiv_id + "'"
        
    md += "\npubnumber: " + pub_number
    
    md += "\ncitation: '" + html_escape(citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page 
    
    if (item.venue == 'preprint'):
        if preprint_type=="biorxiv":
            md += '\n[bioRxiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://www.biorxiv.org/content/10.1101/' + str(item.arxiv_id) + ")\n" 
            md += '[[pdf] <i class="fa fa-download fa-xs" aria-hidden="true"></i>](http://charlieduclut.github.io/files/' + item.url_slug + ".pdf)\n<br/>"       
        elif preprint_type=="arxiv":
            md += '\n[arXiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://arxiv.org/abs/' + str(item.arxiv_id) + ")\n" 
            md += '[[pdf] <i class="fa fa-download fa-xs" aria-hidden="true"></i>](http://charlieduclut.github.io/files/' + item.url_slug + ".pdf)\n<br/>" 
    else: 
        if len(item.paper_url)>1:
            md += '\n[Published version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](' + item.paper_url + ")\n" 
        if len(item.url_slug)>1:
            md += '\n[[pdf] <i class="fa fa-download fa-xs" aria-hidden="true"></i>](http://charlieduclut.github.io/files/' + item.url_slug + ".pdf)\n"
            # md += '<br/>'
        if len(item.arxiv_id)>1:
            if preprint_type=="biorxiv":
                md += '\n[bioRxiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://www.biorxiv.org/content/10.1101/' + str(item.arxiv_id) + ")\n" 
            if preprint_type=="arxiv":
                md += '\n[arXiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://arxiv.org/abs/' + str(item.arxiv_id) + ")\n"      
    # else:
    #     md += '\n[Published version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](' + item.paper_url + ")\n" 
    #     md += '[[pdf] <i class="fa fa-download fa-xs" aria-hidden="true"></i>](http://charlieduclut.github.io/files/' + item.url_slug + ".pdf)\n<br/>"
    #     md += '\n[arXiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://arxiv.org/abs/' + str(item.arxiv_id) + ")\n"      
     
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [62]:
from pathlib import Path
markdown_publi_path = Path("../_publications/")
len(list(markdown_publi_path.glob('*.md')))-len(publications)

0

In [65]:
!ls ../_publications/

2013-06-04-duclut2013hysteresis.md
2016-09-02-benitez2016langevin.md
2017-01-05-duclut2017frequency.md
2017-07-27-duclut2017nonuniversality.md
2019-09-24-duclut2019fluid.md
2021-01-29-mahdisoltani2021nonequilibrium.md
2021-05-11-duclut2021hydraulic.md
2021-09-27-duclut2021nonlinear.md
2022-01-06-benalizinati2022stochastic.md
2022-03-11-duclut2022coarse-grained.md
2022-03-23-duclut2022active.md
2022-05-26-lier2022passive.md
2023-08-02-lier2023lift.md
2023-10-10-delzescaux2023auxiliary.md
2023-10-30-amiri2023random.md
2023-12-20-piscitello-gomez2023core.md
2024-02-13-spera2024nematic.md
2024-04-10-duclut2024probe.md
2024-08-08-martin2024fluctuationinduced.md
2024-08-08-tan2024emergent.md
2024-08-09-fuhrmann2024active.md
2025-04-08-liu2025injuryinduced.md
2025-04-29-shi2025physics.md


In [66]:
!cat ../_publications/2025-04-29-shi2025physics.md

---
title: "Physics of notochord tube expansion in ascidians"
collection: publications
permalink: /publication/2025-04-29-shi2025physics
urlslug: 'shi2025physics'
excerpt: 'Interaction of cells and the surrounding lumen drives the formation of tubular system that plays the transport and exchange functions within an organism. The physical and biological mechanisms of lumen expansion have been explored. However, how cells communicate and coordinate with the surrounding lumen, leading to continuous tube expansion to a defined geometry, is crucial but remains elusive. In this study, we utilized the ascidian notochord tube as a model to address the underlying mechanisms. We firstly quantitively measured and calculated the geometric parameters and found that tube expansion experienced three distinct phases. During the growth processes, we identified and experimentally demonstrated that both Rho GTPase Cdc42 signaling-mediated cell cortex distribution and the stability of tight junctions (TJs

In [21]:
!cat ../_publications/2022-03-11-duclut2022coarse-grained.md

---
title: "Coarse-grained curvature tensor on polygonal surfaces"
collection: publications
permalink: /publication/2022-03-11-duclut2022coarse-grained
urlslug: 'duclut2022coarse-grained'
excerpt: 'Using concepts from integral geometry, we propose a definition for a local coarse-grained curvature tensor that is well-defined on polygonal surfaces. This coarse-grained curvature tensor shows fast convergence to the curvature tensor of smooth surfaces, capturing with accuracy not only the principal curvatures but also the principal directions of curvature. Thanks to the additivity of the integrated curvature tensor, coarse-graining procedures can be implemented to compute it over arbitrary patches of polygons. When computed for a closed surface, the integrated curvature tensor is identical to a rank-2 Minkowski tensor. We also provide an algorithm to extend an existing C++ package, that can be used to compute efficiently local curvature tensors on triangulated surfaces.'
date: 2022-03-11
a